In [1]:
!nvidia-smi

Tue Aug  6 06:12:55 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8             12W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# faster way to get bigger HF models to kaggle, 
# we dodge the storage limitations set by kaggle on the output folder

# mkdir "/kaggle/tmp"
# !cd /kaggle/tmp
!git clone https://github.com/HariWu1995/InstantID-FaceSwap.git

Cloning into 'InstantID-FaceSwap'...
remote: Enumerating objects: 535, done.
remote: Counting objects: 100% (379/379), done.
remote: Compressing objects: 100% (205/205), done.
remote: Total 535 (delta 271), reused 209 (delta 173), pack-reused 156
Receiving objects: 100% (535/535), 154.98 MiB | 47.63 MiB/s, done.
Resolving deltas: 100% (295/295), done.


In [3]:
# !pip install accelerate safetensors omegaconf insightface controlnet_aux
!pip install -r /kaggle/working/InstantID-FaceSwap/gradio_demo/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 2.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 7.7 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 57.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 71.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.0/419.0 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!cd InstantID-FaceSwap

from distutils.dir_util import copy_tree

src_path = "/kaggle/input/antelopev2"
dst_path = "/kaggle/working/InstantID-FaceSwap/models/antelopev2"
copy_tree(src_path, dst_path)

['/kaggle/working/InstantID-FaceSwap/models/antelopev2/glintr100.onnx',
 '/kaggle/working/InstantID-FaceSwap/models/antelopev2/genderage.onnx',
 '/kaggle/working/InstantID-FaceSwap/models/antelopev2/scrfd_10g_bnkps.onnx',
 '/kaggle/working/InstantID-FaceSwap/models/antelopev2/2d106det.onnx',
 '/kaggle/working/InstantID-FaceSwap/models/antelopev2/1k3d68.onnx']

In [5]:
# Provide Ngrok authkey 
# which you can generate freely on https://ngrok.com
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
secret_value = user_secrets.get_secret("ngrok_auth_key")

In [6]:
!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update
!sudo apt-get install -y cuda-drivers

Get:1 https://packages.cloud.google.com/apt gcsfuse-focal InRelease [1227 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:3 https://packages.cloud.google.com/apt cloud-sdk InRelease [1616 B]       
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1581 B]
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]        
Hit:6 https://packages.cloud.google.com/apt google-fast-socket InRelease       
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]      
Get:8 https://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [25.6 kB]
Hit:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease               
Get:10 https://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [3142 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1608 kB]
Get:12 https://packages.cloud.google.com/apt cloud-sdk/main all Pac

In [7]:
!pip install pyngrok

from pyngrok import ngrok
ngrok.set_auth_token(secret_value)

In [8]:
import os
os.chdir("/kaggle/working/InstantID-FaceSwap")

!ls

LICENSE    examples	     models
README.md  gradio_demo	     pipeline_sdxl_instantid.py
assets	   infer.py	     pipeline_sdxl_instantid_full.py
cog	   infer_full.py     pipeline_sdxl_instantid_img2img.py
cog.yaml   infer_img2img.py  pipeline_sdxl_instantid_inpaint.py
docs	   ip_adapter


In [ ]:
import os
import asyncio

# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

async def run_process(cmd, wait: int = None):
    if isinstance(wait, int):
        cmd = ['timeout', str(wait)] + cmd
    print('>>> starting', *cmd)

    p = await asyncio.subprocess.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE,
    )

    async def pipe(lines):
        async for line in lines:
            print(line.strip().decode('utf-8'))

    await asyncio.gather(
        pipe(p.stdout),
        pipe(p.stderr),
    )
    

await asyncio.gather(
    run_process(['python', 'gradio_demo/app_faceswap.py', 
                 '--pretrained_model', '/kaggle/input/yamermix-v8/YamerMIX_v8',
                 '--face_encoder_dir', './',
                 '--face_adapter_path', '/kaggle/input/instantid-sdxl/InstantID/ip-adapter.bin',
                 '--controlnet_path', '/kaggle/input/instantid-sdxl/InstantID/ControlNetModel',
                ]),
    run_process(['ngrok', 'http', '--log', 'stderr', '7860', 
                 '--host-header="localhost:7860"']),
)

>>> starting python gradio_demo/app_faceswap.py --pretrained_model /kaggle/input/yamermix-v8/YamerMIX_v8 --face_encoder_dir ./ --face_adapter_path /kaggle/input/instantid-sdxl/InstantID/ip-adapter.bin --controlnet_path /kaggle/input/instantid-sdxl/InstantID/ControlNetModel
>>> starting ngrok http --log stderr 7860 --host-header="localhost:7860"
t=2024-08-06T06:19:22+0000 lvl=info msg="no configuration paths supplied"
t=2024-08-06T06:19:22+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
t=2024-08-06T06:19:22+0000 lvl=info msg="open config file" path=/root/.config/ngrok/ngrok.yml err=nil
t=2024-08-06T06:19:22+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
t=2024-08-06T06:19:23+0000 lvl=info msg="client session established" obj=tunnels.session
t=2024-08-06T06:19:23+0000 lvl=info msg="tunnel session started" obj=tunnels.session
t=2024-08-06T06:19:23+0000 lvl=info msg="started tunnel" obj=tunnels name=com